In [66]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from numpy import matlib
import seaborn as sns
from PyalData.pyaldata.interval import restrict_to_interval 
from numpy.linalg import eig



In [21]:
orig_df = pd.read_pickle('Test_OutputUnrestricted.pkl')
preparatory_rates = restrict_to_interval(orig_df,"idx_trial_start","idx_go_cue",0,0,None,None,None,False, True, 'motor_cortex_rates')
movement_rates = restrict_to_interval(orig_df,"idx_go_cue","idx_trial_end",0,0,None,None,None,False, True, 'motor_cortex_rates')

In [31]:
p = preparatory_rates.loc[preparatory_rates.target_id == 0]
average = p['motor_cortex_rates'].mean()
average.shape

(81, 50)

PREPARATION
<\n> preprates is an n x ct matrix where n is the number of neurons, c is the number of targets and t is the number of timestamps.

In [33]:

for i in range(8):

    if i == 0:
        preptarget = preparatory_rates.loc[preparatory_rates.target_id == i]
        averagetargetrates = np.transpose(preptarget['motor_cortex_rates'].mean())
        preprates = averagetargetrates
    else:
        preptarget = preparatory_rates.loc[preparatory_rates.target_id == i]
        averagetargetrates = np.transpose(preptarget['motor_cortex_rates'].mean())
        preprates = np.concatenate((preprates, averagetargetrates), axis=1)


preprates.shape

(50, 648)

MOVEMENT
Movementrates is an n x ct matrix where n is the number of neurons, c is the number of targets and t is the number of timestamps.

In [34]:
for i in range(8):

    if i == 0:
        movetarget = movement_rates.loc[movement_rates.target_id == i]
        averagetargetrates =  np.transpose(movetarget['motor_cortex_rates'].mean())
        movementrates = averagetargetrates
    else:
        movetarget = movement_rates.loc[movement_rates.target_id == i]
        averagetargetrates = np.transpose(movetarget['motor_cortex_rates'].mean())
        movementrates = np.concatenate((movementrates, averagetargetrates), axis=1)

movementrates.shape

(50, 648)

FINDING COVARIANCE MATRIX

In [60]:
#Find covariance matrix
mean_prep = np.mean(preprates, axis = 1)
mean_move = np.mean(movementrates, axis = 1)
mean_prep = np.transpose(np.matlib.repmat(mean_prep,648,1))
mean_move = np.transpose(np.matlib.repmat(mean_move,648,1))

transformed_prep = preprates - mean_prep
transformed_move = movementrates - mean_move


covarianceprep = (1/(transformed_prep.shape[0]*transformed_prep.shape[1]))*np.matmul(transformed_prep,np.transpose(transformed_prep))
covariancemove = (1/(transformed_move.shape[0]*transformed_move.shape[1]))*np.matmul(transformed_move,np.transpose(transformed_move))


(50, 648)
(50, 648)
(50, 648)


PRINCIPAL COMPONENTS ANALYSIS

In [70]:
val_prep, pc_prep = eig(covarianceprep)
val_move, pc_move = eig(covariancemove)

print(val_prep)
print(pc_prep.shape)


[9.78351478e-03 4.34054621e-03 2.18770886e-03 2.93815014e-04
 2.58982705e-04 2.07229474e-04 1.01242527e-04 4.72911997e-05
 3.00975789e-05 1.55646740e-05 1.03935345e-05 8.97395512e-06
 6.21201798e-06 5.84426152e-06 4.46668355e-06 3.35420395e-06
 2.37174481e-06 1.58771979e-06 1.13862427e-06 8.15541853e-07
 5.28945293e-07 4.58507429e-07 4.27024048e-07 2.24092375e-07
 1.74355378e-07 1.47354726e-07 1.19346311e-07 8.77681074e-08
 7.26940499e-08 5.56926771e-08 3.87688424e-08 3.83533170e-08
 1.12966962e-11 2.14143316e-08 7.00705050e-09 1.03691171e-08
 1.77159762e-08 1.50347823e-08 1.38705811e-08 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00]
(50, 50)
